In [47]:
import yfinance as yf
import matplotlib.pyplot as plt
import statistics
import numpy as np
from pandas.tseries.offsets import BDay
import pandas as pd
import calendar

# Load data for AMZ
amzn_data = pd.read_csv('stock_data_AMEN.csv')
amzn_data['Date'] = pd.to_datetime(amzn_data['Date'])
amzn_data.set_index('Date', inplace=True)

# Load data for NCLH
nclh_data = pd.read_csv('stock_data.csv')
nclh_data['Date'] = pd.to_datetime(nclh_data['Date'])
nclh_data.set_index('Date', inplace=True)

In [48]:
def lag1_forecast(stock_data, date, N):
    # Filter the stock data up to the specified date
    date = pd.to_datetime(date)
    stock_data_before_date = stock_data[stock_data.index <= date]

    # Calculate the start index
    start_index = len(stock_data_before_date) - N

    # Extract the closing prices within the specified window
    closing_prices = stock_data_before_date['Volume'].iloc[start_index:]

    # Stocks the price at date (the one we want to forcast), removes it from the list
    real_price = closing_prices[-1]
    closing_prices = closing_prices[:-1]

    # Create the lagged version of the closing prices (y(t-1))
    closing_prices_lag1 = closing_prices.shift(1)
    closing_prices = closing_prices[1:]  # Remove the first element to align with lag 1

    # retrieves the closing price at date, to later on forecast the price at date "date"
    Y_1 = closing_prices[-1]

    # Perform linear regression
    y = closing_prices
    X = np.column_stack((closing_prices_lag1[1:], np.ones(len(y))))

    # Calculate coefficients using numpy.linalg.lstsq
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]

    A, B = coeffs

    # Calculate the residuals (ε)
    residuals = y - (A * closing_prices_lag1[1:] + B)
    residual = statistics.mean(residuals)

    # Calculate the forcasted price with this method:
    forecasted_volume = A*Y_1 + B + residual

    return forecasted_volume
    

def get_real_volume(stock_data, date):
    date = pd.to_datetime(date)
    # Calculate real volume
    real_volume = stock_data.loc[stock_data.index == date]['Volume']
    # Check if real_volume is empty
    if real_volume.empty:
        return 0  # No trading volume available for this date, return 0
    else:
        return real_volume.iloc[0]  # Return the actual traded volume if available


def lag2_forecast(stock_data, date, N):
    # Filter the stock data up to the specified date
    date = pd.to_datetime(date)
    stock_data_before_date = stock_data[stock_data.index <= date]

    # Calculate the start index
    start_index = len(stock_data_before_date) - N

    # Extract the closing prices within the specified window
    closing_prices = stock_data_before_date['Volume'].iloc[start_index:]
    
    # Stocks the price at date (the one we want to forecast), removes it from the list
    real_price = closing_prices[-1]
    closing_prices = closing_prices[:-1]

    # Create the lagged versions of the closing prices (y(t-1) and y(t-2))
    closing_prices_lag1 = closing_prices.shift(1)
    closing_prices_lag2 = closing_prices.shift(2)
    closing_prices = closing_prices[2:]  # Remove the first two elements to align with lag 1 and lag 2

    # Retrieves the closing price at date and day before date to later forecast the price at date "date"
    Y_1 = closing_prices[-1]
    Y_2 = closing_prices[-2]

    # Perform linear regression
    y = closing_prices
    X = np.column_stack((closing_prices_lag1[2:], closing_prices_lag2[2:], np.ones(len(y))))

    # Calculate coefficients using numpy.linalg.lstsq
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    A, B, C = coeffs

    # Calculate the residuals (ε)
    residuals = y - (A * closing_prices_lag1[2:] + B * closing_prices_lag2[2:] + C)
    residual = statistics.mean(residuals)

    # Calculate the forecasted price with this method:
    forecasted_volume = A * Y_1 + B * Y_2 + C + residual
    return forecasted_volume


In [49]:
def get_monthly_MSE(year, stock_data, N, forecast_type):
    monthly_MSE_list = []

    for month in range(1, 13):  # Iterate through each month
        # Calculate the first day of the month
        start_date = pd.to_datetime(f"{year}-{month:02d}-01")
        end_date = start_date + pd.DateOffset(days=calendar.monthrange(year, month)[1] - 1)

        # Initialize the lists
        forecast_list = []
        real_volume_list = []

        # We forecast each day of the month using the 'forecast_type' method:
        for day in pd.date_range(start_date, end_date):
            if forecast_type == 1:
                forecast = lag1_forecast(stock_data, day, N)
            elif forecast_type == 2:
                forecast = lag2_forecast(stock_data, day, N)

            real_volume = get_real_volume(stock_data, day)
            
            # Skip the day if real volume is 0
            if real_volume == 0:
                continue

            forecast_list.append(forecast)
            real_volume_list.append(real_volume)

        # Calculate mean squared error
        if len(real_volume_list) > 0:  # Check if there are days with real volume
            MSE = sum((x - y) ** 2 for x, y in zip(forecast_list, real_volume_list)) / len(real_volume_list)
            monthly_MSE_list.append(MSE)

    return monthly_MSE_list

year = 2019  
forecast_type = 2  
stock_data = amzn_data 
N = 10

monthly_MSE = get_monthly_MSE(year, stock_data, N, forecast_type)
print(monthly_MSE)

[1451526208239555.5, 2212700936597187.8, 2269860445820817.0, 2254020821603643.5, 334837432952673.56, 2136670879043708.5, 501821888960859.25, 477050995596192.75, 741712929150507.2, 1.1120970852558094e+16, 408377200880460.2, 1480414761769042.8]


In [50]:
def get_yearly_MSE(year, stock_data, N, forecast_type):
    monthly_MSE = get_monthly_MSE(year, stock_data, N, forecast_type)
    return np.mean(monthly_MSE)

year = 2019  
forecast_type = 2  
stock_data = amzn_data 
N = 10
get_yearly_MSE(year, stock_data, N, forecast_type)

2115830446097728.8

In [51]:
# ------------------- forecaste the N+1st day ----------------------

stock_data = amzn_data
lag1 =[]
lag2 = []
real_list= []

lag1.append(lag1_forecast(stock_data, '2019-01-16', 10))
lag1.append(lag1_forecast(stock_data, '2019-03-01', 30))
lag1.append(lag1_forecast(stock_data, '2019-04-12', 60))

lag2.append(lag2_forecast(stock_data, '2019-01-16', 10))
lag2.append(lag2_forecast(stock_data, '2019-03-01', 30))
lag2.append(lag2_forecast(stock_data, '2019-04-12', 60))

real_list.append(get_real_volume(stock_data, '2019-01-16'))
real_list.append(get_real_volume(stock_data, '2019-03-01'))
real_list.append(get_real_volume(stock_data, '2019-04-12'))

print(lag1)
print(lag2)
print(real_list)


stock_data = amzn_data
lag1 =[]
lag2 = []
real_list= []

lag1.append(lag1_forecast(stock_data, '2020-01-15', 10))
lag1.append(lag1_forecast(stock_data, '2020-02-14', 30))
lag1.append(lag1_forecast(stock_data, '2020-03-30', 60))

lag2.append(lag2_forecast(stock_data, '2020-01-15', 10))
lag2.append(lag2_forecast(stock_data, '2020-02-14', 30))
lag2.append(lag2_forecast(stock_data, '2019-03-30', 60))

real_list.append(get_real_volume(stock_data, '2020-01-15'))
real_list.append(get_real_volume(stock_data, '2020-02-14'))
real_list.append(get_real_volume(stock_data, '2020-03-30'))

print(lag1)
print(lag2)
print(real_list)

[129043619.54877537, 73800580.6179587, 69352037.19294089]
[116674890.45554996, 75667745.58359289, 70035043.42154223]
[127338000, 99498000, 62288000]
[67922195.93298924, 75834172.15940754, 109282995.08824311]
[80486673.8841565, 77314642.67192203, 76804386.32924885]
[57932000, 52124000, 122522000]


In [52]:
def fill_amzn_table(years, n_values,forecast_type):
    amzn_table = []
    for year in years:
        year_row = [year]
        for n in n_values:
            mse = get_yearly_MSE(year, amzn_data, n, forecast_type)
            year_row.extend([f"{mse:.4e}"])
        amzn_table.append(year_row)
    return amzn_table

# Function to fill the table for NCLH
def fill_nclh_table(years, n_values,forecast_type):
    nclh_table = []
    for year in years:
        year_row = [year]
        for n in n_values:
            mse= get_yearly_MSE(year, nclh_data, n, forecast_type)
            year_row.extend([f"{mse:.4e}"])
        nclh_table.append(year_row)
    return nclh_table

# Define the years and N values
years = [2017, 2018, 2019, 2020]
n_values = [10, 30, 60]


# Fill the tables
amzn_table1 = fill_amzn_table(years, n_values,1)
amzn_table2 = fill_amzn_table(years, n_values,2)
nclh_table1 = fill_nclh_table(years, n_values,1)
nclh_table2 = fill_nclh_table(years, n_values,2)


# Print the tables
print("Table 1: Average Mean Square Error (MSE) for Amazon lag 1")
print(amzn_table1)
# Print the tables
print("Table 1: Average Mean Square Error (MSE) for Amazon lag 2")
print(amzn_table2)

print("\nTable 2: Average Mean Square Error (MSE) for NCLH lag 1")
print(nclh_table1)
print("\nTable 2: Average Mean Square Error (MSE) for NCLH lag 2")
print(nclh_table2)

Table 1: Average Mean Square Error (MSE) for Amazon lag 1
[[2017, '6.1200e+15', '1.9903e+15', '1.2035e+15'], [2018, '2.1817e+15', '1.4232e+15', '1.3694e+15'], [2019, '1.2901e+15', '6.6736e+14', '6.2881e+14'], [2020, '3.1660e+15', '1.1627e+15', '1.1665e+15']]
Table 1: Average Mean Square Error (MSE) for Amazon lag 2
[[2017, '1.1647e+16', '2.1911e+15', '1.2216e+15'], [2018, '3.5829e+15', '1.5768e+15', '1.4147e+15'], [2019, '2.1158e+15', '7.1278e+14', '6.5929e+14'], [2020, '7.2396e+15', '1.4502e+15', '1.3012e+15']]

Table 2: Average Mean Square Error (MSE) for NCLH lag 1
[[2017, '9.5911e+12', '6.2522e+12', '5.5509e+12'], [2018, '3.3527e+12', '1.7710e+12', '1.7713e+12'], [2019, '9.1992e+11', '6.1704e+11', '6.0309e+11'], [2020, '2.7982e+14', '2.3027e+14', '2.3852e+14']]

Table 2: Average Mean Square Error (MSE) for NCLH lag 2
[[2017, '1.8438e+13', '6.5282e+12', '5.5976e+12'], [2018, '8.1788e+12', '2.2675e+12', '1.8112e+12'], [2019, '1.3703e+12', '6.1347e+11', '5.9761e+11'], [2020, '3.6144e+

In [55]:
# -------------------- holidays -----------------------
# Define a list of holidays for 2019
holidays_2019 = [
    "2019-10-14",  # Columbus Day
    "2019-11-11",  # Veteran's Day
    "2019-11-29",  # Friday after Thanksgiving
    "2019-12-31",  # December 31
]


# Define a function to calculate the MSE
def calculate_mse(stock_data, date, N, forecast_type):
    # Calculate forecast and get the real price for the given date
    forecast = 0
    real_price = 0  # Replace with the actual real price for the date

    if forecast_type == 1:
        # Calculate the average forecast
        forecast = lag1_forecast(stock_data, date, N)
    elif forecast_type == 2:
        # Calculate the median forecast
        forecast = lag2_forecast(stock_data, date, N)

    # Calculate the Mean Squared Error
    mse = (forecast - real_price) ** 2
    return mse

# Function to calculate the MSE for a given forecast type
def calculate_mse_for_holidays(stock_data, holidays, N, forecast_type):
    mse_values = []
    for holiday_date in holidays:
        mse = calculate_mse(stock_data, holiday_date, N, forecast_type)
        mse_values.append(mse)
    return mse_values

stock_data = amzn_data 

# Calculate MSE for Average and Median forecasts for N=10
mse_lag1 = calculate_mse_for_holidays(stock_data, holidays_2019, N=10, forecast_type=1)
mse_lag2 = calculate_mse_for_holidays(stock_data, holidays_2019, N=10, forecast_type=2)

# Print or store the MSE values as needed
print("MSE for lag 1 (N=10):", mse_lag1)
print("MSE for lag2 (N=10):", mse_lag2)





MSE for lag 1 (N=10): [3523658174630054.0, 2280866780155061.5, 3242103638923186.5, 5638390300038662.0]
MSE for lag2 (N=10): [3643184853021432.5, 2059846930327505.2, 3134683639220781.5, 1287825541021742.5]


In [60]:

twzo_first_months = (get_monthly_MSE(2019, amzn_data, N = 10, forecast_type=1)[0] + get_monthly_MSE(2019, amzn_data, N = 10, forecast_type=1)[1])/2
print(f"{twzo_first_months:.4e}")



1.4372e+15
